****Import Table Extractor****
* Extract imported functions (only includes functions linked at compile time)

*This is just a dumb piece of code, we didn't give any particular attention to the beauty, the optimization or the conventions of python when writing this notebook. If you have any remarks about that please leave a comment we'd love to enhance our coding skills.*

In [ ]:
!pip install pefile

In [ ]:
import sys
import pefile
import pandas as pd
import os.path
from os import listdir
import pprint
import json

In [ ]:
malware_base_dir = '../input/firstpart/samples/'
exceptions_counter = 0
counter = 1
exceptions = []
malware_list = pd.read_csv('../input/malwarelist/malware_list_sample_1.csv', index_col='Hash Of Malware')
rows = {}
a = 1
for f in listdir(malware_base_dir):
    if not f.endswith('.json') and malware_list.loc[f]['Status']=='clear':
        counter += 1
        exe = pefile.PE(os.path.join(malware_base_dir,f))
        rows[f] = {}
        try:
            for entry in exe.DIRECTORY_ENTRY_IMPORT:
                rows[f][entry.dll.decode('utf-8')]=[]
                for func in entry.imports:
                    if func.name is not None:
                        rows[f][entry.dll.decode('utf-8')].append({
                            'name':func.name.decode('utf-8'), 'ordinal':func.ordinal
                        })
                    else:
                        rows[f][entry.dll.decode('utf-8')].append({
                            'name':None, 'ordinal':func.ordinal
                        })
        except:
            exceptions.append([f,'No imports'])
        print(counter)

In [ ]:
with open('imports.json', 'a') as imports:
    json.dump(rows,imports)